In [ ]:
import maboss
import ginsim
import pandas as pd 
import numpy as np
import mygene
import os
import shutil
import ast
from scipy.stats import kruskal

from create_generic_models.create_generic_patients_cfgs import create_generic_patients_cfg_bnd_validation
from create_generic_models.update_phenotypes_generic_models import generic_models_update_phenotypes


# from create_person_models.tailor_cfgs_patients_gene import personalized_patients_genes_cfgs_validation
from pre_process_data.pre_process_genes import create_table_rna_seq_patients, create_table_proteins_patients

from identification_patients.validation_get_patients_ids import get_patients_valid

from pre_process_data.tcga_preprocess_data import pre_process_tcga_data

from create_person_models.tailor_cfgs_patients_gene import personalized_patients_genes_cfgs, personalized_patients_proteins_cfgs
from create_person_models.tailor_bnd_cnv import tailor_bnd_cnv_validation

from create_person_models.tailor_bnd_tsg_onco_mutations import tailor_bnd_mutat_validation

from MaBoSS_simulation.maboss_phenotype_patient import compute_phenotype_table, compute_phenotype_mean_group_validation, combine_groups_values
from stats.stats_proba import compute_kruskal_test_means_validation

In [2]:
# Import data
phenotype_data = pd.read_csv('data/TCGA_data/prostate/TCGA_PRAD_phenotypes.csv')
genes_data = pd.read_csv('data/TCGA_data/prostate/TCGA_PRAD_genes_illumina.csv', sep='\t')
cnv_data = pd.read_csv('data/TCGA_data/prostate/TCGA_PRAD_cnv_gistic2.csv',sep='\t')

proteins_data = pd.read_csv('data/TCGA_data/prostate/TCGA_PRAD_proteins_RPPA.csv',sep='\t')

# keep all montagud nodes
montagud_data = (
    pd.read_csv('data/Montagud_inter_nodes_data.csv', header=1)
    .loc[:, ['Target node', 'Interaction type', 'Source']])

# Create list of genes of interest (in Montagud data)
montagud_nodes = list(set(montagud_data['Target node'].tolist() + montagud_data['Source'].tolist()))
montagud_nodes = [node for node in montagud_nodes if node != '0/1']
montagud_nodes = [node.upper() for node in montagud_nodes if isinstance(node, str)]
# montagud_nodes.append('KRAS')
# to_remove = ['RAS', 'FUSED_EVENT', 'NKX3_1', 'SPOP', 'AR_ERG']

# montagud_nodes = [node for node in montagud_nodes if node not in to_remove]
# montagud_nodes = list(set(montagud_nodes))

In [3]:
# remove all '_'
montagud_nodes = ['MEK1' if x == 'MEK1_2' else x for x in montagud_nodes]
montagud_nodes = ['TSC1' if x == 'TSC1_2' else x for x in montagud_nodes]
montagud_nodes = ['MAP3K1' if x == 'MAP3K1_3' else x for x in montagud_nodes]
montagud_nodes = ['CHK1' if x == 'CHK1_2' else x for x in montagud_nodes]


montagud_nodes.append('MEK2')
montagud_nodes.append('TSC2')
montagud_nodes.append('MAP3K3')
montagud_nodes.append('CHK2')


to_remove = ['FUSED_EVENT', 'AR_ERG']

montagud_nodes = [node for node in montagud_nodes if node not in to_remove]
print(montagud_nodes)

['PTCH1', 'BAD', 'ETV1', 'CYCLINB', 'GSK3', 'ETS1', 'APOPTOSIS', 'CFLAR', 'EP300', 'TAK1', 'BRCA1', 'EGFR', 'MIGRATION', 'RAS', 'PKC', 'TGFBR', 'RTK', 'E2F1', 'ROS', 'P90RSK', 'BRCA2', 'CHK1', 'P53', 'TWIST1', 'DAXX', 'METASTASIS', 'GLUT1', 'APAF1', 'P38', 'RAGS', 'PDK1', 'PI3K', 'AMP_ATP', 'GLI', 'DSH', 'HYPOXIA', 'EEF2K', 'EGF', 'FADD', 'TCF', 'RHEB', 'COX4I2', 'FGF', 'IKK', 'FOXA1', 'MEK1', 'ZBTB17', 'VHL', 'CASPASE9', 'MDM2', 'MAP3K1', 'SLUG', 'WNT', 'NCOA3', 'EZH2', 'TSC1', 'PROLIFERATION', 'INVASION', 'TNFALPHA', 'BIRC5', 'NF1', 'AXIN1', 'BCL_XL', 'CYTOC', 'PTEN', 'MTORC1', 'P70S6KAB', 'PHDS', 'ATR', 'SMAD', 'AR', 'CASPASE8', 'MYC', 'BMP2', 'MYC_MAX', 'P21', 'NUTRIENTS', 'EEF2', 'TGFB', 'CYCLIND', 'MXI1', 'CDH2', 'GADD45', 'JUN', 'AMPK', 'EMT', 'E_CADHERIN', 'PIP3', 'VEGF', 'DNA_DAMAGE', 'HSPS', 'CASPASE3', 'BETA_CATENIN', 'P15', 'BAK', 'CARCINOGEN', 'SMO', 'NF_KB', 'JNK', 'NKX3_1', 'RB1', 'ERK', 'GSH', 'LACTIC_ACID', 'BAX', 'RAF', 'FGFR3', 'NCOR2', 'ANDROGEN', 'SHH', 'NCOR1', 'F

In [4]:
# print(proteins_data.head())
# print(list(proteins_data['sample']))
proteins_data['sample'] = proteins_data['sample'].str.rsplit('-', n=2).str[0]
proteins_data['sample'] = proteins_data['sample'].str.upper()

# Create regex pattern that matches any name in the list
pattern = '|'.join(montagud_nodes)  # 'NameA|NameB|NameC'

# Filter rows where 'sample' contains any name from the list
proteins_data = proteins_data[proteins_data['sample'].str.contains(pattern, case=False, na=False)]
proteins_data = proteins_data.dropna(how='all', subset=proteins_data.columns[1:])


print(proteins_data)



          sample  TCGA-KK-A59Z-01  TCGA-KC-A4BV-01  TCGA-CH-5772-01  \
8    A-RAF_PS299         0.121288        -0.167044        -0.110590   
13         ADAR1         0.371003        -0.209629        -0.086770   
14           AKT         0.237545        -0.434174        -0.174623   
15     AKT_PS473        -1.609986        -0.246830        -0.105235   
16     AKT_PT308        -1.462328        -0.261684        -0.325260   
..           ...              ...              ...              ...   
219          BAK              NaN              NaN              NaN   
220  C-RAF_PS338              NaN              NaN              NaN   
221         CHK1              NaN              NaN              NaN   
222    CHK2_PT68              NaN              NaN              NaN   
226        PARP1              NaN              NaN              NaN   

     TCGA-G9-6377-01  TCGA-HC-7081-01  TCGA-EJ-7788-01  TCGA-HC-7233-01  \
8           0.066131        -0.223209         0.309472         0.177667 

AKT: how much AKT protein is present.
AKT_PS473: how much of the protein is in the activated or modified form.

how to do with the activated form of the protein ? should i combine it with the full complex?

In [5]:

# diagnoses.tumor_grade, diagnoses.morphology
# stratify by cancer type: diagnoses.primary_diagnosis

phenotype_data_filtered = phenotype_data[['sampleID','gleason_score']]


In [6]:
print(phenotype_data_filtered)
print(phenotype_data_filtered['gleason_score'].value_counts())

            sampleID  gleason_score
0    TCGA-2A-A8VL-01              6
1    TCGA-2A-A8VO-01              6
2    TCGA-2A-A8VT-01              9
3    TCGA-2A-A8VV-01              6
4    TCGA-2A-A8VX-01              8
..               ...            ...
561  TCGA-ZG-A9M4-01              9
562  TCGA-ZG-A9MC-01              9
563  TCGA-ZG-A9N3-01              9
564  TCGA-ZG-A9ND-01              9
565  TCGA-ZG-A9NI-01              9

[566 rows x 2 columns]
gleason_score
7     301
9     144
8      67
6      50
10      4
Name: count, dtype: int64


In [7]:
# create 3 groups: gleason score of 6, gleason score of 7, and of gleason score of > 8

group_0 = [6]
group_1 = [7]
group_2 = [8, 9, 10]

conditions = [
phenotype_data_filtered["gleason_score"].isin(group_0),
phenotype_data_filtered["gleason_score"].isin(group_1),
phenotype_data_filtered["gleason_score"].isin(group_2),

]
choices = ['low_aggressive', 'middle_aggressive', 'high_aggressive']

phenotype_data_filtered.loc[:, "Gleason_group"] = np.select(
conditions, choices, default=""
)



/var/folders/83/pqwsm5cn7vscbq8l3fnf30qr0000gn/T/ipykernel_71036/1549429481.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phenotype_data_filtered.loc[:, "Gleason_group"] = np.select(


In [8]:
print(phenotype_data_filtered)
sampled_df = phenotype_data_filtered.groupby("Gleason_group", group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), 30), random_state=42)
)

            sampleID  gleason_score    Gleason_group
0    TCGA-2A-A8VL-01              6   low_aggressive
1    TCGA-2A-A8VO-01              6   low_aggressive
2    TCGA-2A-A8VT-01              9  high_aggressive
3    TCGA-2A-A8VV-01              6   low_aggressive
4    TCGA-2A-A8VX-01              8  high_aggressive
..               ...            ...              ...
561  TCGA-ZG-A9M4-01              9  high_aggressive
562  TCGA-ZG-A9MC-01              9  high_aggressive
563  TCGA-ZG-A9N3-01              9  high_aggressive
564  TCGA-ZG-A9ND-01              9  high_aggressive
565  TCGA-ZG-A9NI-01              9  high_aggressive

[566 rows x 3 columns]


/var/folders/83/pqwsm5cn7vscbq8l3fnf30qr0000gn/T/ipykernel_71036/4059254103.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = phenotype_data_filtered.groupby("Gleason_group", group_keys=False).apply(


In [9]:
print(sampled_df)
patients_id = list(sampled_df['sampleID'])
print(patients_id)

            sampleID  gleason_score      Gleason_group
551  TCGA-ZG-A9L5-01              9    high_aggressive
563  TCGA-ZG-A9N3-01              9    high_aggressive
464  TCGA-V1-A9ZK-01              8    high_aggressive
525  TCGA-YL-A8SF-01              8    high_aggressive
52   TCGA-CH-5792-01              9    high_aggressive
..               ...            ...                ...
382  TCGA-KK-A6E5-01              7  middle_aggressive
392  TCGA-KK-A7AZ-01              7  middle_aggressive
110  TCGA-EJ-7330-01              7  middle_aggressive
166  TCGA-EJ-A8FN-01              7  middle_aggressive
165  TCGA-EJ-A7NN-01              7  middle_aggressive

[90 rows x 3 columns]
['TCGA-ZG-A9L5-01', 'TCGA-ZG-A9N3-01', 'TCGA-V1-A9ZK-01', 'TCGA-YL-A8SF-01', 'TCGA-CH-5792-01', 'TCGA-KK-A8I9-01', 'TCGA-YL-A8HK-01', 'TCGA-HC-A48F-01', 'TCGA-ZG-A9LM-01', 'TCGA-V1-A9ZR-01', 'TCGA-XJ-A9DX-01', 'TCGA-EJ-8472-01', 'TCGA-YL-A9WI-01', 'TCGA-CH-5761-01', 'TCGA-HC-7821-01', 'TCGA-YL-A8SC-01', 'TCGA-YL-A8S

In [10]:
cnv_data_col = list(cnv_data.columns)
common_col = list(set(cnv_data_col) & set(patients_id))
col_keep = ['Gene Symbol'] + common_col
cnv_data_filtered = cnv_data[col_keep]
print(cnv_data_filtered.head())

  Gene Symbol  TCGA-YL-A9WH-01  TCGA-HC-7748-01  TCGA-V1-A9OT-01  \
0       ACAP3               -1                0                0   
1      ACTRT2               -1                0                0   
2        AGRN               -1                0                0   
3     ANKRD65               -1                0                0   
4      ATAD3A               -1                0                0   

   TCGA-EJ-AB20-01  TCGA-HC-7736-01  TCGA-HC-8260-01  TCGA-HC-8258-01  \
0                0                0                0                0   
1                0                0                0                0   
2                0                0                0                0   
3                0                0                0                0   
4                0                0                0                0   

   TCGA-J4-A83N-01  TCGA-CH-5738-01  TCGA-J4-A67Q-01  TCGA-VP-A87D-01  \
0                0                0                0                0   
1     

In [11]:
df_melted_cnv = cnv_data_filtered.melt(
    id_vars=["Gene Symbol"],       # columns to keep fixed
    var_name="samples_id",         # name for the variable column (sample IDs)
    value_name="expression_value"  # name for the values
)


df_melted_cnv['Gene Symbol'] = df_melted_cnv['Gene Symbol'].str.split('|').str[0] 


df_melted_cnv = df_melted_cnv.rename(
    columns={
        "samples_id": "model_id",
        "Gene Symbol": "gene_symbol",
        "expression_value": "rsem_tpm",
    }
)

print(df_melted_cnv.head())


  gene_symbol         model_id  rsem_tpm
0       ACAP3  TCGA-YL-A9WH-01        -1
1      ACTRT2  TCGA-YL-A9WH-01        -1
2        AGRN  TCGA-YL-A9WH-01        -1
3     ANKRD65  TCGA-YL-A9WH-01        -1
4      ATAD3A  TCGA-YL-A9WH-01        -1


In [12]:
group_loss = [-1, -2]
group_normal = [0]
group_gain = [1, 2]

conditions = [
    df_melted_cnv["rsem_tpm"].isin(group_loss),
    df_melted_cnv["rsem_tpm"].isin(group_normal),
    df_melted_cnv["rsem_tpm"].isin(group_gain),
]
choices = ["Loss", "Normal", "Gain"]
df_melted_cnv.loc[:, "effect"] = np.select(conditions, choices, default="")

df_melted_cnv = df_melted_cnv[df_melted_cnv['gene_symbol'].isin(montagud_nodes)]
df_melted_cnv.to_csv('data/TCGA_data/prostate/filtered_data/cnv_samples_table.csv')

In [13]:
# list(df_melted_cnv['gene_symbol'].unique())
# df_melted_cnv[df_melted_cnv['gene_symbol'] == 'BCL2L1']

In [14]:
proteins_data

,sample,TCGA-KK-A59Z-01,TCGA-KC-A4BV-01,TCGA-CH-5772-01,TCGA-G9-6377-01,TCGA-HC-7081-01,TCGA-EJ-7788-01,TCGA-HC-7233-01,TCGA-G9-6378-01,TCGA-CH-5761-01,TCGA-G9-6494-01,TCGA-G9-6366-01,TCGA-HC-7210-01,TCGA-FC-A4JI-01,TCGA-EJ-7315-01,TCGA-EJ-5501-01,TCGA-EJ-7314-01,TCGA-G9-6338-01,TCGA-KK-A5A1-01,TCGA-CH-5767-01,TCGA-G9-7521-01,TCGA-HC-7077-01,TCGA-HC-7817-01,TCGA-G9-6371-01,TCGA-CH-5743-01,TCGA-KK-A59Y-01,TCGA-G9-7525-01,TCGA-KC-A4BN-01,TCGA-CH-5750-01,TCGA-EJ-A46D-01,TCGA-EJ-5522-01,TCGA-J9-A52C-01,TCGA-EJ-7321-01,TCGA-CH-5766-01,TCGA-HC-7079-01,TCGA-EJ-7125-01,TCGA-G9-6370-01,TCGA-HC-7213-01,TCGA-J4-8198-01,TCGA-G9-6356-01,TCGA-FC-7708-01,TCGA-H9-7775-01,TCGA-EJ-5506-01,TCGA-EJ-A46G-01,TCGA-HI-7168-01,TCGA-EJ-5526-01,TCGA-HC-7740-01,TCGA-CH-5794-01,TCGA-CH-5788-01,TCGA-EJ-5516-01,TCGA-EJ-5519-01,TCGA-EJ-5530-01,TCGA-EJ-7115-01,TCGA-CH-5753-01,TCGA-KK-A59X-01,TCGA-G9-6351-01,TCGA-HC-7736-01,TCGA-CH-5763-01,TCGA-EJ-A46B-01,TCGA-EJ-5503-01,TCGA-EJ-5525-01,TCGA-G9-6364-01,TCGA-EJ-5518-01,TCGA-CH-5737-01,TCGA-J4-8200-01,TCGA-EJ-7792-01,TCGA-G9-7510-01,TCGA-EJ-7330-01,TCGA-G9-6362-01,TCGA-G9-6384-01,TCGA-CH-5738-01,TCGA-EJ-7317-01,TCGA-G9-6365-01,TCGA-CH-5789-01,TCGA-EJ-5498-01,TCGA-EJ-8474-01,TCGA-EJ-5515-01,TCGA-FC-7961-01,TCGA-G9-7523-01,TCGA-EJ-7785-01,TCGA-EJ-5499-01,TCGA-HC-7748-01,TCGA-EJ-5512-01,TCGA-CH-5771-01,TCGA-EJ-5517-01,TCGA-EJ-7328-01,TCGA-HC-7744-01,TCGA-EJ-A65M-01,TCGA-EJ-5508-01,TCGA-HI-7170-01,TCGA-CH-5740-01,TCGA-EJ-7793-01,TCGA-KC-A4BO-01,TCGA-CH-5752-01,TCGA-EJ-5504-01,TCGA-CH-5739-01,TCGA-EJ-5511-01,TCGA-EJ-5521-01,TCGA-EJ-7791-01,TCGA-HC-7212-01,TCGA-CH-5769-01,TCGA-EJ-A65E-01,TCGA-G9-6343-01,TCGA-G9-6373-01,TCGA-EJ-7789-01,TCGA-HC-7211-01,TCGA-CH-5751-01,TCGA-CH-5754-01,TCGA-G9-6363-01,TCGA-KK-A59V-01,TCGA-CH-5748-01,TCGA-EJ-5496-01,TCGA-G9-7522-01,TCGA-EJ-5524-01,TCGA-EJ-5509-01,TCGA-EJ-5531-01,TCGA-EJ-5514-01,TCGA-G9-6353-01,TCGA-EJ-7331-01,TCGA-CH-5762-01,TCGA-EJ-7797-01,TCGA-EJ-5542-01,TCGA-EJ-5494-01,TCGA-HC-8264-01,TCGA-EJ-5507-01,TCGA-HC-7232-01,TCGA-CH-5790-01,TCGA-J9-A52E-01,TCGA-EJ-A46E-01,TCGA-HI-7171-01,TCGA-CH-5744-01,TCGA-CH-5741-01,TCGA-EJ-5527-01,TCGA-CH-5768-01,TCGA-CH-5792-01,TCGA-EJ-5497-01,TCGA-EJ-5495-01,TCGA-G9-6342-01,TCGA-EJ-8470-01,TCGA-G9-6361-01,TCGA-CH-5791-01,TCGA-G9-6499-01,TCGA-G9-6496-01,TCGA-CH-5764-01,TCGA-EJ-8469-01,TCGA-G9-6367-01,TCGA-FC-A5OB-01,TCGA-EJ-7784-01,TCGA-KC-A4BL-01,TCGA-EJ-7781-01,TCGA-EJ-A46F-01,TCGA-EJ-7783-01,TCGA-EJ-8468-01,TCGA-EJ-7782-01,TCGA-EJ-7786-01,TCGA-HI-7169-01,TCGA-EJ-5532-01,TCGA-EJ-5502-01,TCGA-G9-6329-01,TCGA-KC-A4BR-01,TCGA-CH-5765-01,TCGA-EJ-7794-01,TCGA-EJ-5510-01,TCGA-EJ-5505-01,TCGA-CH-5745-01,TCGA-2A-A8VL-01,TCGA-V1-A8MU-01,TCGA-KK-A8I6-01,TCGA-KK-A8I8-01,TCGA-YL-A8SL-01,TCGA-YL-A8SF-01,TCGA-EJ-AB20-01,TCGA-EJ-A8FS-01,TCGA-G9-A9S4-01,TCGA-EJ-A7NN-01,TCGA-J9-A8CP-01,TCGA-EJ-A7NK-01,TCGA-J4-A83N-01,TCGA-EJ-A6RA-01,TCGA-EJ-A8FO-01,TCGA-KK-A8IF-01,TCGA-ZG-A9LY-01,TCGA-YL-A8SH-01,TCGA-VP-AA1N-01,TCGA-YL-A8SA-01,TCGA-J4-A67Q-01,TCGA-EJ-7325-01,TCGA-V1-A9OX-01,TCGA-KK-A7AP-01,TCGA-2A-A8W3-01,TCGA-G9-6498-01,TCGA-M7-A71Z-01,TCGA-HC-A76X-01,TCGA-KK-A8IC-01,TCGA-KC-A7F5-01,TCGA-VN-A88K-01,TCGA-V1-A8MF-01,TCGA-YL-A8S8-01,TCGA-FC-A66V-01,TCGA-YL-A8SI-01,TCGA-KK-A7B4-01,TCGA-TP-A8TV-01,TCGA-YL-A9WI-01,TCGA-J4-A6M7-01,TCGA-G9-6339-01,TCGA-2A-A8VT-01,TCGA-EJ-A7NJ-01,TCGA-KK-A7AU-01,TCGA-KK-A8IJ-01,TCGA-ZG-A8QX-01,TCGA-2A-A8VO-01,TCGA-KK-A7B3-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QY-01,TCGA-TP-A8TT-01,TCGA-KK-A8IA-01,TCGA-YL-A8SQ-01,TCGA-EJ-A65D-01,TCGA-YL-A8HM-01,TCGA-TK-A8OK-01,TCGA-VP-A87E-01,TCGA-YL-A9WY-01,TCGA-YL-A8SC-01,TCGA-VN-A88I-01,TCGA-KC-A7F3-01,TCGA-SU-A7E7-01,TCGA-J4-A67O-01,TCGA-ZG-A9LB-01,TCGA-J4-A67K-01,TCGA-EJ-A8FN-01,TCGA-G9-A9S7-01,TCGA-J4-A67M-01,TCGA-ZG-A9LS-01,TCGA-J4-A67N-01,TCGA-KK-A8IB-01,TCGA-M7-A720-01,TCGA-KK-A6E0-01,TCGA-EJ-A6RC-01,TCGA-ZG-A9MC-01,TCGA-YL-A9WL-01,TCGA-ZG-A9L4-01,TCGA-VP-A878-01,TCGA-YL-A8SR-01,TCGA-V1-A9OL-01,TCGA-KC-A7FD-01,TCGA-YL-A9WX-01,TCGA-2A-AAYO-01,TCGA-ZG-A9L9-01,TCGA-KK-A8ID-01,TCGA-EJ-7312-01,TCGA-ZG-

In [15]:
# do with proteins instead of genes

proteins_data_col = list(proteins_data.columns)
common_col = list(set(proteins_data_col) & set(patients_id))
col_keep = ['sample'] + common_col
proteins_data_filtered = proteins_data[col_keep]


df_melted_protein = proteins_data_filtered.melt(
    id_vars=["sample"],       # columns to keep fixed
    var_name="samples_id",         # name for the variable column (sample IDs)
    value_name="expression_value"  # name for the values
)

df_melted_protein['sample'] = df_melted_protein['sample'].str.split('|').str[0] 

df_melted_protein = df_melted_protein.rename(
    columns={
        "samples_id": "model_id",
        "sample": "protein_symbol",
        "expression_value": "rsem_tpm",
    }
)
df_melted_protein['protein_symbol'] = df_melted_protein['protein_symbol'].str.upper()
# df_melted_protein['protein_symbol'] = df_melted_protein['protein_symbol'].str.replace('_', '', regex=False)
print(df_melted_protein)

# df_melted_protein.to_csv('data/TCGA_data/prostate/filtered_data/proteins_samples_table.csv')

     protein_symbol         model_id  rsem_tpm
0       A-RAF_PS299  TCGA-CH-5743-01  0.007295
1             ADAR1  TCGA-CH-5743-01 -0.087798
2               AKT  TCGA-CH-5743-01 -0.018654
3         AKT_PS473  TCGA-CH-5743-01  1.990607
4         AKT_PT308  TCGA-CH-5743-01  1.873595
...             ...              ...       ...
4083            BAK  TCGA-EJ-A7NN-01 -0.123457
4084    C-RAF_PS338  TCGA-EJ-A7NN-01  0.122859
4085           CHK1  TCGA-EJ-A7NN-01 -0.194214
4086      CHK2_PT68  TCGA-EJ-A7NN-01 -0.027627
4087          PARP1  TCGA-EJ-A7NN-01 -0.207367

[4088 rows x 3 columns]


In [16]:
print(df_melted_protein['protein_symbol'].unique())

['A-RAF_PS299' 'ADAR1' 'AKT' 'AKT_PS473' 'AKT_PT308' 'AMPK_ALPHA'
 'AMPK_PT172' 'AR' 'DIRAS3' 'ATM' 'B-RAF' 'BAD_PS112' 'BAK' 'BAX' 'BRCA2'
 'C-JUN_PS73' 'C-MYC' 'C-RAF' 'C-RAF_PS338' 'CHK1' 'CHK1_PS345' 'CHK2'
 'CHK2_PT68' 'EEF2' 'EEF2K' 'EGFR' 'EGFR_PY1068' 'EGFR_PY1173' 'ERK2'
 'FOXO3A' 'FOXO3A_PS318_S321' 'GSK3-ALPHA-BETA' 'GSK3-ALPHA-BETA_PS21_S9'
 'GSK3_PS9' 'JNK_PT183_PY185' 'JNK2' 'MEK1' 'MEK1_PS217_S221' 'N-RAS'
 'P21' 'P38_MAPK' 'P38_PT180_Y182' 'P53' 'P90RSK' 'P90RSK_PT359_S363'
 'PDK1' 'PDK1_PS241' 'PI3K-P110-ALPHA' 'PI3K-P85' 'PKC-ALPHA'
 'PKC-ALPHA_PS657' 'PKC-DELTA_PS664' 'PKC-PAN_BETAII_PS660' 'PRAS40_PT246'
 'PTEN' 'SMAD1' 'SMAD3' 'SMAD4' 'TIGAR' 'TSC1' 'VEGFR2' 'PARP_CLEAVED'
 'SNAIL' 'A-RAF' 'ARID1A' 'B-RAF_PS445' 'BCL2A1' 'CHK1_PS296' 'PARP1']


In [17]:
# print(df_melted_protein['protein_symbol'].value_counts())

print(df_melted_protein)
test_df = df_melted_protein[df_melted_protein['protein_symbol'] == 'CHK2']
num_rows = test_df.shape[0]


     protein_symbol         model_id  rsem_tpm
0       A-RAF_PS299  TCGA-CH-5743-01  0.007295
1             ADAR1  TCGA-CH-5743-01 -0.087798
2               AKT  TCGA-CH-5743-01 -0.018654
3         AKT_PS473  TCGA-CH-5743-01  1.990607
4         AKT_PT308  TCGA-CH-5743-01  1.873595
...             ...              ...       ...
4083            BAK  TCGA-EJ-A7NN-01 -0.123457
4084    C-RAF_PS338  TCGA-EJ-A7NN-01  0.122859
4085           CHK1  TCGA-EJ-A7NN-01 -0.194214
4086      CHK2_PT68  TCGA-EJ-A7NN-01 -0.027627
4087          PARP1  TCGA-EJ-A7NN-01 -0.207367

[4088 rows x 3 columns]


In [18]:

# print(df_melted_protein['protein_symbol'].value_counts())
# print(df_melted_protein['protein_symbol'].unique())


protein_list = list(df_melted_protein['protein_symbol'])

mods = ['_PS', '_PT', '_PY']

df_melted_protein = df_melted_protein[
    ~df_melted_protein['protein_symbol'].apply(lambda p: any(mod in p for mod in mods))
]
print(df_melted_protein)
# replace each name by the corresponding name in the montagud nodes list  



     protein_symbol         model_id  rsem_tpm
1             ADAR1  TCGA-CH-5743-01 -0.087798
2               AKT  TCGA-CH-5743-01 -0.018654
5        AMPK_ALPHA  TCGA-CH-5743-01 -0.040532
7                AR  TCGA-CH-5743-01  0.563090
8            DIRAS3  TCGA-CH-5743-01 -0.081379
...             ...              ...       ...
4079         ARID1A  TCGA-EJ-A7NN-01 -0.214734
4081         BCL2A1  TCGA-EJ-A7NN-01  0.283037
4083            BAK  TCGA-EJ-A7NN-01 -0.123457
4085           CHK1  TCGA-EJ-A7NN-01 -0.194214
4087          PARP1  TCGA-EJ-A7NN-01 -0.207367

[2576 rows x 3 columns]


In [19]:
def replace_with_base_name(protein_name):
    for base in montagud_nodes:
        if protein_name.startswith(base):
            return base
    return protein_name  # if no match found, keep original

# Assuming your dataframe is df and column to replace is 'protein_symbol'
df_melted_protein['protein_symbol'] = df_melted_protein['protein_symbol'].apply(replace_with_base_name)

In [20]:
df_melted_protein = df_melted_protein[df_melted_protein['protein_symbol'].isin(montagud_nodes)]
df_melted_protein['protein_symbol'] = df_melted_protein['protein_symbol'].str.replace('_', '', regex=False)
df_melted_protein = df_melted_protein[df_melted_protein['rsem_tpm'].notna()]
df_melted_protein.to_csv('data/TCGA_data/prostate/filtered_data/proteins_samples_table.csv')

In [21]:
# pre-process tcga data based on montagud nodes and patients ids
df_melted_cnv= pd.read_csv('data/TCGA_data/prostate/filtered_data/cnv_samples_table.csv')
def_melted_proteins = pd.read_csv('data/TCGA_data/prostate/filtered_data/proteins_samples_table.csv')

In [22]:
def_melted_proteins = def_melted_proteins[['protein_symbol', 'model_id', 'rsem_tpm']]
def_melted_proteins

,protein_symbol,model_id,rsem_tpm
0,AKT,TCGA-CH-5743-01,-0.018654
1,AMPK,TCGA-CH-5743-01,-0.040532
2,AR,TCGA-CH-5743-01,0.563090
3,ATM,TCGA-CH-5743-01,-0.070493
4,BAK,TCGA-CH-5743-01,-0.035079
...,...,...,...
1899,SNAIL,TCGA-EJ-A7NN-01,-0.223618
1900,AR,TCGA-EJ-A7NN-01,-0.214734
1901,BCL2,TCGA-EJ-A7NN-01,0.283037
1902,BAK,TCGA-EJ-A7NN-01,-0.123457


In [23]:
 # Create generic models 

folder_generic_models_cfg = 'validation/prostate/generic_models/proteins_models/Montagud2022_Prostate_Cancer.cfg'
folder_generic_models_bnd = 'validation/prostate/generic_models/proteins_models/Montagud2022_Prostate_Cancer.bnd'

folder_pers_models = 'validation/prostate/personalized_models/proteins_models'

tissue = 'Prostate'

create_generic_patients_cfg_bnd_validation(folder_generic_models_cfg, folder_generic_models_bnd, folder_pers_models, patients_id, tissue)


All .cfg and .bnd files created for the validation.


In [24]:
# update phenotypes in generic models 

phenotype_interest = ["Proliferation","Invasion","DNA_Repair","Migration","Apoptosis"]
original_data_dir = "validation/prostate/personalized_models/proteins_models"
results_dir = "validation/prostate/personalized_models/proteins_models"


generic_models_update_phenotypes(phenotype_interest, original_data_dir, results_dir)


Updated PTCH1.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated BAD.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated ETV1.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated GSK3.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated P14ARF.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated ETS1.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated Apoptosis.is_internal=0 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated EEF2K.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated EP300.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated TAK1.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated BRCA1.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated EGFR.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated Migration.is_internal=0 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated EEF2.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated P90RSK.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated RAS.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated PKC.is_internal=1 in TCGA-ZG-A9L

In [25]:
# personalize the boolean networks with genes 
table_proteins_patients = create_table_proteins_patients(def_melted_proteins)
print(table_proteins_patients.head())

protein_expression_level             High Protein Abundance  \
model_id                                                      
TCGA-2A-A8VL-01                          AR, P21, PKC, VEGF   
TCGA-2A-AAYO-01           BAX, BRCA2, EEF2, ERK, MEK1, PI3K   
TCGA-2A-AAYU-01                                AKT, AR, BAX   
TCGA-CH-5738-01                                        SMAD   
TCGA-CH-5743-01                                    AR, SMAD   

protein_expression_level             Low Protein Abundance  
model_id                                                    
TCGA-2A-A8VL-01                                       PI3K  
TCGA-2A-AAYO-01                                       PTEN  
TCGA-2A-AAYU-01                          EEF2K, PI3K, SMAD  
TCGA-CH-5738-01           AR, BAK, EEF2K, FOXO, MEK1, PDK1  
TCGA-CH-5743-01                            PKC, SMAD, TSC1  


/Users/romane/repos/icr/Attractor-Resistance/pre_process_data/pre_process_genes.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proteins_data.rename(columns={"protein_symbol": "protein_name"}, inplace=True)


In [26]:
personalized_patients_proteins_cfgs(df_melted_protein,montagud_nodes,folder_pers_models,folder_pers_models,patients_id,table_proteins_patients,tissue)

Modified and saved: validation/prostate/personalized_models/proteins_models/TCGA-ZG-A9L5-01_Prostate.cfg
Modified and saved: validation/prostate/personalized_models/proteins_models/TCGA-EJ-A7NN-01_Prostate.bnd
Modified and saved: validation/prostate/personalized_models/proteins_models/TCGA-G9-6343-01_Prostate.bnd
Modified and saved: validation/prostate/personalized_models/proteins_models/TCGA-CH-5743-01_Prostate.bnd
Modified and saved: validation/prostate/personalized_models/proteins_models/TCGA-YL-A9WI-01_Prostate.bnd
Modified and saved: validation/prostate/personalized_models/proteins_models/TCGA-YL-A8SF-01_Prostate.bnd
Modified and saved: validation/prostate/personalized_models/proteins_models/TCGA-Y6-A8TL-01_Prostate.bnd
Modified and saved: validation/prostate/personalized_models/proteins_models/TCGA-V1-A8MF-01_Prostate.bnd
Modified and saved: validation/prostate/personalized_models/proteins_models/TCGA-YL-A8S8-01_Prostate.cfg
Modified and saved: validation/prostate/personalized_mo

In [27]:
# personalize with CNV
tailor_bnd_cnv_validation(df_melted_cnv, original_data_dir, tissue)

🔍 Processing patient TCGA-YL-A9WH-01, gene: ZBTB17
Patient TCGA-YL-A9WH-01 is in both gain and loss groups. Please review.
ZBTB17 node found. Replacing...
🔍 Processing patient TCGA-YL-A9WH-01, gene: JUN
Patient TCGA-YL-A9WH-01 is in both gain and loss groups. Please review.
JUN node found. Replacing...
🔍 Processing patient TCGA-YL-A9WH-01, gene: PDK1
Patient TCGA-YL-A9WH-01 is in both gain and loss groups. Please review.
PDK1 node found. Replacing...
🔍 Processing patient TCGA-YL-A9WH-01, gene: CFLAR
Patient TCGA-YL-A9WH-01 is in both gain and loss groups. Please review.
CFLAR node found. Replacing...
🔍 Processing patient TCGA-YL-A9WH-01, gene: IDH1
Patient TCGA-YL-A9WH-01 is in both gain and loss groups. Please review.
IDH1 node found. Replacing...
🔍 Processing patient TCGA-YL-A9WH-01, gene: VHL
Patient TCGA-YL-A9WH-01 is in both gain and loss groups. Please review.
VHL node found. Replacing...
🔍 Processing patient TCGA-YL-A9WH-01, gene: ATR
Patient TCGA-YL-A9WH-01 is in both gain and 

In [28]:
# # add mutations info 

# # Add somatic mutations data 
# # keep only the one we had for the main pipeline

# mutations_data = pd.read_csv('data/TCGA_data/TCGA_mutations_mutect2_GDC-PANCAN.csv')
# mutations_data_filtered = mutations_data[mutations_data['Sample_ID'].isin(patients_id)]
# mutations_data_filtered = mutations_data_filtered[mutations_data_filtered['gene'].isin(montagud_nodes)]

# # check if genes are TSG/ Oncogenes
# onco_tsg_gene = pd.read_csv('data/unknown_origin/oncogenes_tsg.csv')
# onco_tsg_gene = onco_tsg_gene[['Hugo Symbol', 'Is Oncogene', 'Is Tumor Suppressor Gene']]
# onco_tsg_gene_filtered = onco_tsg_gene[onco_tsg_gene['Hugo Symbol'].isin(montagud_nodes)]
# onco_tsg_gene_filtered = onco_tsg_gene_filtered.rename(columns={'Hugo Symbol': 'gene'})
# # oncogenes = onco_tsg_gene_filtered[onco_tsg_gene_filtered['Is Oncogene'] == 'Yes']
# # tsg_genes = onco_tsg_gene_filtered[onco_tsg_gene_filtered['Is Tumor Suppressor Gene'] == 'Yes']


# mutations_annotated = mutations_data_filtered.merge(
#     onco_tsg_gene_filtered[['gene', 'Is Oncogene', 'Is Tumor Suppressor Gene']],
#     on='gene',
#     how='left'
# )
# mutations_annotated = mutations_annotated.rename(columns={'Is Oncogene': 'oncogene', 'Is Tumor Suppressor Gene': 'tsg'})

# mutations_annotated = mutations_annotated[
#     mutations_annotated['oncogene'].notna() | mutations_annotated['tsg'].notna()
# ]

# # loss function mutation assumption -> TSG and 
# lof_effects = ["frameshift_variant", "stop_gained", "start_lost", "splice_region_variant"]
# lof_mutations = mutations_annotated[mutations_annotated['effect'].isin(lof_effects)]
# lof_mutations_tsg = lof_mutations[(lof_mutations['tsg'] == 'Yes') & (lof_mutations['oncogene'] == 'No')]
# lof_mutations_tsg_filtered = lof_mutations_tsg[['Sample_ID', 'gene']]


# mutations_onco = mutations_annotated[(mutations_annotated['tsg'] == 'No') & (mutations_annotated['oncogene'] == 'Yes')]
# print(mutations_onco.head())
# # dna_vaf > 0.5 -> clonal mutation (mutation probably in the early tumor cells)
# # gof_effects = ['p.G12D','p.S249C', 'p.Y373C']
# # gof_mutations = mutations_onco[mutations_onco['Amino_Acid_Change'].isin(gof_effects)]
# # gof_mutations_filtered = gof_mutations[['Sample_ID', 'gene']]




Let's try to add more info to reflect metastasis change, add mutations data

In [29]:
# tailor_bnd_mutat_validation(lof_mutations_tsg_filtered,gof_mutations_filtered,folder_pers_models, tissue)

In [30]:

folder_models = "validation/prostate/personalized_models/proteins_models"
folder_save_results = "validation/prostate/results/proteins_models/phenotype_distribution/phenotype_table"
phenotypes_interest = [
    "Proliferation",
    "Invasion",
    "DNA_Repair",
    "Migration",
    "Apoptosis",
]

inputs_list = [
    "EGF",
    "FGF",
    "TGFB",
    "Androgen",
    "Hypoxia",
    "Nutrients",
    "Carcinogen",
    "Acidosis",
    "TNF",
    "fused_event",
    "SPOP",
]
for patient in patients_id:
    compute_phenotype_table(folder_save_results,folder_models,patient,inputs_list,phenotypes_interest,tissue="Prostate")


In [31]:
phenotype_data_filtered.head()
# print(phenotype_data_filtered['Gleason_group'].unique())

,sampleID,gleason_score,Gleason_group
0,TCGA-2A-A8VL-01,6,low_aggressive
1,TCGA-2A-A8VO-01,6,low_aggressive
2,TCGA-2A-A8VT-01,9,high_aggressive
3,TCGA-2A-A8VV-01,6,low_aggressive
4,TCGA-2A-A8VX-01,8,high_aggressive


In [45]:
low_group_ids= list(phenotype_data_filtered[phenotype_data_filtered['Gleason_group'] == 'low_aggressive']['sampleID'])
medium_group_ids= list(phenotype_data_filtered[phenotype_data_filtered['Gleason_group'] == 'middle_aggressive']['sampleID'])
high_group_ids= list(phenotype_data_filtered[phenotype_data_filtered['Gleason_group'] == 'high_aggressive']['sampleID'])

In [ ]:
#move each files to directory corresponding

# Map group names to sample ID lists
group_mapping = {
    "low_group": low_group_ids,
    "medium_group": medium_group_ids,
    "high_group": high_group_ids,
}

# Folder where all the files currently are
source_dir = "validation/prostate/results/proteins_models/phenotype_distribution/phenotype_table"

dest_base_dir = "validation/prostate/results/proteins_models/phenotype_group_means"

# Loop over all files in the source directory
for filename in os.listdir(source_dir):
    if not filename.startswith("_TCGA"):
        continue
    sample_id = filename.replace("_", "").replace(".csv", "")
    # Determine the group of this sample
    group_found = False
    for group_name, id_list in group_mapping.items():
        if sample_id in id_list:
            group_folder = os.path.join(dest_base_dir, group_name)
            os.makedirs(group_folder, exist_ok=True)

            src_path = os.path.join(source_dir, filename)
            dst_path = os.path.join(group_folder, filename)
            shutil.move(src_path, dst_path)

            group_found = True
            break

    if not group_found:
        print(f" Sample ID {sample_id} not found in any group list.")


In [46]:
# combine all the values
groups = ["low_group", "medium_group", "high_group"]

folder_groups_means = "validation/prostate/results/proteins_models/phenotype_group_means"
# to do-> change name (only compute mean not validation)
mean_df =compute_phenotype_mean_group_validation(groups, folder_groups_means)

              Proliferation  Invasion  DNA_Repair  Migration  Apoptosis
Acidosis           0.292123  0.035813    0.233113   0.011228   0.072441
Androgen           0.300313  0.060190    0.232747   0.040647   0.068539
Carcinogen         0.333286  0.076200    0.444396   0.012908   0.186672
EGF                0.317034  0.071839    0.235267   0.032093   0.105377
FGF                0.331681  0.060549    0.233520   0.012204   0.136078
Hypoxia            0.253374  0.034519    0.233728   0.018652   0.063748
Nutrients          0.356842  0.038546    0.232668   0.016591   0.059296
SPOP               0.335883  0.099491    0.232835   0.029573   0.046258
TGFB               0.261204  0.266426    0.233843   0.029161   0.220895
TNF                0.266355  0.289102    0.231578   0.066811   0.071465
fused_event        0.292009  0.039256    0.233135   0.015191   0.064952
Overall_Mean       0.303646  0.097448    0.252439   0.025915   0.099611
              Proliferation  Invasion  DNA_Repair  Migration  Ap

In [51]:
# combine values of a directory together

from collections import defaultdict
def collect_group_data(group_folder_path):
    combined_data = defaultdict(lambda: defaultdict(list))

    for file in os.listdir(group_folder_path):
        if file.startswith("_TCGA") and file.endswith(".csv"):
            file_path = os.path.join(group_folder_path, file)
            df = pd.read_csv(file_path, index_col=0)

            for input_name in df.index:
                for phenotype in df.columns:
                    value = df.at[input_name, phenotype]
                    combined_data[input_name][phenotype].append(float(value))

    result_df = pd.DataFrame.from_dict(combined_data, orient='index')
    result_df.to_csv(os.path.join(group_folder_path, "combined_results.csv"))

    return result_df


base_path = "validation/prostate/results/proteins_models/phenotype_group_means"

group_names = ["low_group", "medium_group", "high_group"]
group_dataframes = {}

for group in group_names:
    folder_path = os.path.join(base_path, group)
    group_df = collect_group_data(folder_path)
    group_dataframes[group] = group_df
group_dataframes

{'low_group':                                                  Proliferation  \
 EGF          [0.024, 0.988, 0.006348, 0.271145, 0.014464, 0...   
 FGF          [0.0238579999999999, 0.98, 0.0289299999999999,...   
 TGFB         [0.02, 0.982598, 0.03, 0.0547869999999999, 0.0...   
 Androgen     [0.0238579999999999, 0.98, 0.0289299999999999,...   
 Hypoxia      [0.016596, 0.98, 0.025499, 0.044189, 0.027192,...   
 Nutrients    [0.012, 0.99, 0.018, 0.413065, 0.020482, 0.966...   
 Carcinogen   [0.017558, 0.984, 0.023612, 0.346861, 0.014, 0...   
 Acidosis     [0.004, 0.98, 0.008, 0.1714709999999999, 0.004...   
 TNF          [0.042, 0.992, 0.0236909999999999, 0.086129, 0...   
 fused_event  [0.021046, 0.989613, 0.019942, 0.155647, 0.044...   
 SPOP         [0.026, 0.975878, 0.0160319999999999, 0.34937,...   
 
                                                       Invasion  \
 EGF          [0.024, 0.013049, 0.010454, 0.2258129999999999...   
 FGF          [0.033914, 0.014, 0.021196, 0.157

In [52]:


# Paths to your combined data CSVs
base_path = "validation/prostate/results/proteins_models/phenotype_group_means"
group_files = {
    "low": os.path.join(base_path, "low_group", "combined_results.csv"),
    "medium": os.path.join(base_path, "medium_group", "combined_results.csv"),
    "high": os.path.join(base_path, "high_group", "combined_results.csv"),
}

# Load all groups into dict of DataFrames
group_dfs = {}
for group, path in group_files.items():
    # Because each cell is a list saved as a string, parse it back to list
    df = pd.read_csv(path, index_col=0)
    # Convert strings like '[1.2, 3.4]' back to Python lists using ast.literal_eval
    df = df.applymap(ast.literal_eval)
    group_dfs[group] = df

# Get all inputs and phenotypes from one dataframe (assuming all share the same shape)
inputs = group_dfs["low"].index
phenotypes = group_dfs["low"].columns

# Prepare result storage
kruskal_results = pd.DataFrame(index=inputs, columns=phenotypes)

# Run Kruskal-Wallis test for each (input, phenotype)
for input_name in inputs:
    for phenotype in phenotypes:
        data_low = group_dfs["low"].at[input_name, phenotype]
        data_medium = group_dfs["medium"].at[input_name, phenotype]
        data_high = group_dfs["high"].at[input_name, phenotype]

        # Run the Kruskal-Wallis test only if all groups have data
        if data_low and data_medium and data_high:
            stat, pvalue = kruskal(data_low, data_medium, data_high)
            kruskal_results.at[input_name, phenotype] = pvalue
        else:
            kruskal_results.at[input_name, phenotype] = None

# Optionally, save the p-values table to CSV
kruskal_results.to_csv(os.path.join(base_path, "kruskal_pvalues.csv"))
print(kruskal_results)


            Proliferation  Invasion DNA_Repair Migration Apoptosis
EGF              0.017723  0.240162   0.017666  0.243646  0.812392
FGF              0.000639  0.005279   0.006998  0.013387  0.004204
TGFB             0.001575  0.001585   0.001145   0.09046  0.123084
Androgen         0.000512  0.002059   0.002614  0.010682  0.002969
Hypoxia          0.000819  0.000626   0.000136  0.026098  0.179885
Nutrients        0.000747  0.001884   0.000129  0.345769  0.690985
Carcinogen       0.002631  0.251649   0.000395  0.208729   0.07889
Acidosis         0.000458  0.016182   0.001118  0.079435  0.015019
TNF              0.009474  0.339702   0.004658  0.097586  0.216672
fused_event      0.007635  0.000707   0.000323  0.009554  0.114082
SPOP             0.000945  0.282215   0.000137  0.122191  0.082015


/var/folders/83/pqwsm5cn7vscbq8l3fnf30qr0000gn/T/ipykernel_71036/1666062943.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(ast.literal_eval)
/var/folders/83/pqwsm5cn7vscbq8l3fnf30qr0000gn/T/ipykernel_71036/1666062943.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(ast.literal_eval)
/var/folders/83/pqwsm5cn7vscbq8l3fnf30qr0000gn/T/ipykernel_71036/1666062943.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(ast.literal_eval)


In [53]:
from statsmodels.stats.multitest import multipletests

# Flatten p-values to a 1D array, ignoring None or NaNs
pvals = kruskal_results.values.flatten()
pvals = [p for p in pvals if p is not None]

# Adjust using BH method
_, pvals_adj, _, _ = multipletests(pvals, alpha=0.05, method='fdr_bh')

# Now, you need to put adjusted p-values back into the DataFrame shape
# Create a copy to fill
adjusted_df = kruskal_results.copy()

# Fill adjusted p-values sequentially where there was a non-None p-value
idx = 0
for i in adjusted_df.index:
    for j in adjusted_df.columns:
        if adjusted_df.at[i, j] is not None:
            adjusted_df.at[i, j] = pvals_adj[idx]
            idx += 1
print(adjusted_df)


            Proliferation  Invasion DNA_Repair Migration Apoptosis
EGF              0.027851   0.27348   0.027851   0.27348  0.812392
FGF              0.003733  0.011614   0.014803  0.023751  0.010053
TGFB             0.005127  0.005127   0.004198  0.124383  0.153855
Androgen         0.003733  0.005961    0.00689  0.019583  0.007422
Hypoxia          0.003752  0.003733    0.00252  0.039872   0.21986
Nutrients        0.003733  0.005756    0.00252  0.358817  0.703781
Carcinogen        0.00689  0.276813   0.003733  0.249567  0.114972
Acidosis         0.003733   0.02697   0.004198  0.114972  0.025815
TNF               0.01812  0.358817   0.010675  0.130908  0.253553
fused_event      0.015552  0.003733   0.003733   0.01812  0.149394
SPOP             0.003996  0.304349    0.00252  0.153855  0.115662


In [54]:
# keep only the significant results
significant_df = adjusted_df.copy()
significant_df[significant_df >= 0.05] = np.nan
significant_df

,Proliferation,Invasion,DNA_Repair,Migration,Apoptosis
EGF,0.027851,NaN,0.027851,NaN,NaN
FGF,0.003733,0.011614,0.014803,0.023751,0.010053
TGFB,0.005127,0.005127,0.004198,NaN,NaN
Androgen,0.003733,0.005961,0.00689,0.019583,0.007422
Hypoxia,0.003752,0.003733,0.00252,0.039872,NaN
Nutrients,0.003733,0.005756,0.00252,NaN,NaN
Carcinogen,0.00689,NaN,0.003733,NaN,NaN
Acidosis,0.003733,0.02697,0.004198,NaN,0.025815
TNF,0.01812,NaN,0.010675,NaN,NaN
fused_event,0.015552,0.003733,0.003733,0.01812,NaN
